In [4]:
library(Seurat)
library(SingleCellSignalR)


Attaching SeuratObject



In [8]:
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install("SingleCellSignalR")

In [5]:
pbmc <- readRDS('/home/jmitchel/data/lupus_data/lupus_subsetted_seurat_v3.rds')

# converting shorthand cell type names to full names
new_names <- sapply(as.character(pbmc@meta.data$cg_cov), function(x){
  if (x=='cM') {
    return('cMono')
  } else if (x=='ncM') {
    return('ncMono')
  } else if (x=='T4') {
    return('Th')
  } else if (x=='T8') {
    return('Tc')
  } else {
    return(x)
  }
})
names(new_names) <- NULL
pbmc@meta.data$cg_cov <- factor(new_names,levels=unique(new_names))


# subset data to SLE patients only
cells_keep <- rownames(pbmc@meta.data)[pbmc@meta.data$Status=='Managed']
pbmc <- subset(pbmc,cells = cells_keep)

# reduce to cells of the cell types that I'm using
cells_keep <- rownames(pbmc@meta.data)[pbmc@meta.data$cg_cov %in% c("B","NK","Th","Tc","cDC","cMono","ncMono")]

In [6]:
cells_keep <- sample(cells_keep,50000)
pbmc <- subset(pbmc,cells = cells_keep)
print(dim(pbmc))

[1] 32738 50000


In [7]:
pbmc@meta.data$cg_cov <- factor(pbmc@meta.data$cg_cov,levels=c("B","NK","Th","Tc","cDC","cMono","ncMono"))
pbmc@meta.data$idents <- pbmc@meta.data$cg_cov
Idents(pbmc) <- pbmc@meta.data$idents
pbmc@meta.data$cell_type <- pbmc@meta.data$cg_cov

In [8]:
# make cluster numbers 
clust_num <- sapply(as.character(pbmc@meta.data$cg_cov), function(x){
  if (x=='cMono') {
    return(1)
  } else if (x=='ncMono') {
    return(2)
  } else if (x=='Th') {
    return(3)
  } else if (x=='Tc') {
    return(4)
  } else if (x=='B') {
    return(5)
  } else if (x=='NK') {
    return(6)
  } else if (x=='cDC') {
    return(7)
  }
})

In [9]:
names(clust_num) <- NULL

In [17]:
# clust_num[1] <- 0

In [10]:
print(length(clust_num))
print(length(pbmc@meta.data$cg_cov))

[1] 50000
[1] 50000


In [ ]:
data <- pbmc@assays$RNA@data
clust.ana <- cluster_analysis(data = data, genes = rownames(data), cluster = clust_num, c.names = c('cMono','ncMono','Th','Tc','B','NK','cDC'), write = FALSE)
